### BASELINE RESULTS MEDSAM

In [ ]:

%timeit import skimage.io

In [ ]:
!pip install git+https://github.com/bowang-lab/MedSAM.git
!git clone https://github.com/bowang-lab/MedSAM
# %% environment and functions
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
import torch
import skimage
import skimage.io
from skimage import transform
import sys
from segment_anything import sam_model_registry
from tabulate import tabulate
import torch.nn.functional as F
#from PIL import Image
import random
import cv2
import numpy as np
from tqdm import tqdm
import time

import random
import cv2
import numpy as np
import torch
from skimage import transform
import os
from tqdm import tqdm
import time
import matplotlib.pyplot as plt


In [ ]:
%matplotlib widget
from segment_anything import sam_model_registry
from MedSAM.utils.demo import BboxPromptDemo
import os

# Define the checkpoint path
checkpoint_folder = '/kaggle/input/medsam/dataset_complete/MedSAM/medsam_vit_b'
checkpoint_file = 'medsam_vit_b.pth'
MedSAM_CKPT_PATH = os.path.join(checkpoint_folder, checkpoint_file)

device = "cuda:0"

medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
medsam_model = medsam_model.to(device)
medsam_model.eval()

In [ ]:


def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251/255, 252/255, 30/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='blue', facecolor=(0,0,0,0), lw=2))


@torch.no_grad()
def medsam_inference(medsam_model, img_embed, box_1024, H, W):
    if box_1024 is not None:
        box_torch = torch.as_tensor(box_1024, dtype=torch.float, device=img_embed.device)
        if len(box_torch.shape) == 2:
            box_torch = box_torch[:, None, :]  # (B, 1, 4)

        sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
            points=None,
            boxes=box_torch,
            masks=None,
        )
    else:
        sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
        
    low_res_logits, _ = medsam_model.mask_decoder(
        image_embeddings=img_embed,  # (B, 256, 64, 64)
        image_pe=medsam_model.prompt_encoder.get_dense_pe(),  # (1, 256, 64, 64)
        sparse_prompt_embeddings=sparse_embeddings,  # (B, 2, 256)
        dense_prompt_embeddings=dense_embeddings,  # (B, 256, 64, 64)
        multimask_output=False,
    )

    low_res_pred = torch.sigmoid(low_res_logits)  # (1, 1, 256, 256)

    low_res_pred = F.interpolate(
        low_res_pred,
        size=(H, W),
        mode="bilinear",
        align_corners=False,
    )  # (1, 1, gt.shape)
    low_res_pred = low_res_pred.squeeze().cpu().numpy()  # (H, W)
    medsam_seg = (low_res_pred > 0.5).astype(np.uint8)
    return medsam_seg

In [ ]:
#evaluation 

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Define input and mask folders for the test set
input_folder = r'C:\Users\cinth\Documentos\ams\data_science\actual_thesis\codes\MedSAM_Universeg_2024\datasets\data\dataset_complete_2\partitioned_dataset_original\images\test'
mask_folder = r'C:\Users\cinth\Documentos\ams\data_science\actual_thesis\codes\MedSAM_Universeg_2024\datasets\data\dataset_complete_2\partitioned_dataset_original\masks\test'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Sort images and masks based on numerical index
images = sorted(os.listdir(input_folder), key=lambda x: int(x.split('_')[-1].split('.')[0]))
masks = sorted(os.listdir(mask_folder), key=lambda x: int(x.split('_')[-2]))

# Initialize lists to store scores
dice_scores = []
jaccard_scores = []
sensitivity_scores = []
precision_scores = []
accuracy_scores = []

# Initialize list to store inference times
inference_times = []

# Select some images to visualize the predictions
num_visualizations = 3
visualized_images = []


for idx, (image_name, mask_name) in enumerate(tqdm(zip(images, masks), total=len(images))):
    # Load images
    image_path = os.path.join(input_folder, image_name)
    image = cv2.imread(image_path)

    # Load ground truth data
    mask_path = os.path.join(mask_folder, mask_name)
    ground_truth = cv2.imread(mask_path)

    # Convert the mask to grayscale
    ground_truth_gray = cv2.cvtColor(ground_truth, cv2.COLOR_BGR2GRAY)

    # Threshold the grayscale mask to separate areas of interest (ROI) from the background
    _, mask = cv2.threshold(ground_truth_gray, 1, 255, cv2.THRESH_BINARY)

    # Convert image to 1024x1024 and normalize
    if len(image.shape) == 2:
        img_3c = np.repeat(image[:, :, None], 3, axis=-1)  # convert to RGB
    else:
        img_3c = image
    H, W, _ = img_3c.shape
    img_1024 = transform.resize(img_3c, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(np.uint8)  
    img_1024 = (img_1024 - img_1024.min()) / np.clip(
        img_1024.max() - img_1024.min(), a_min=1e-8, a_max=None
    )  # normalize to [0, 1], (H, W, 3)
    img_1024_tensor = torch.tensor(img_1024).float().permute(2, 0, 1).unsqueeze(0).to(device)

    overall_segmentation = np.zeros((H, W), dtype=np.uint8)

    if np.any(mask):
        # Process images except for those with background label only
        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate through contours and create bounding box for each contour
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)

            # Define margin
            margin = 10

            # Expand the bounding box by the margin
            x_margin = max(x - margin, 0)
            y_margin = max(y - margin, 0)
            w_margin = min(w + 2 * margin, image.shape[1] - x_margin)
            h_margin = min(h + 2 * margin, image.shape[0] - y_margin)

            # Convert the expanded bounding box coordinates to the format used in MedSAM (x_min, y_min, x_max, y_max)
            box_np = np.array([[x_margin, y_margin, x_margin + w_margin, y_margin + h_margin]])

            # Transfer box_np to 1024x1024 scale
            box_1024 = box_np / np.array([W, H, W, H]) * 1024  # bounding box scaled to 1024x1024

            # Measure inference time
            start_time = time.time()

            # Perform inference with MedSAM model
            with torch.no_grad():
                image_embedding = medsam_model.image_encoder(img_1024_tensor)  
            medsam_seg = medsam_inference(medsam_model, image_embedding, box_1024, H, W)  

            end_time = time.time()
            inference_time = end_time - start_time
            inference_times.append(inference_time)

            # Resize the segmentation mask back to the original image size
            medsam_seg_resized = transform.resize(medsam_seg, (H, W), order=0, preserve_range=True).astype(np.uint8)
            overall_segmentation = np.maximum(overall_segmentation, medsam_seg_resized)

        # Calculate metrics directly
        segmentation_mask = (overall_segmentation > 0).astype(np.uint8)

        # Compute intersection and union of the masks
        intersection = np.logical_and(ground_truth_gray > 0, segmentation_mask > 0)
        union = np.logical_or(ground_truth_gray > 0, segmentation_mask > 0)

        if np.sum(ground_truth_gray > 0) == 0 and np.sum(segmentation_mask > 0) == 0:
            # Special case: both ground truth and segmentation are completely background
            dice_score = 1.0
            jaccard_score = 1.0
            sensitivity = 1.0
            precision = 1.0
            accuracy = 1.0
        else:
            # Compute Dice score
            dice_score = 2 * np.sum(intersection) / (np.sum(ground_truth_gray > 0) + np.sum(segmentation_mask > 0))

            # Compute Jaccard index (IoU)
            jaccard_score = np.sum(intersection) / np.sum(union)

            # Compute sensitivity (recall)
            true_positives = np.sum(intersection)
            false_negatives = np.sum((ground_truth_gray > 0) & (segmentation_mask == 0))
            sensitivity = true_positives / (true_positives + false_negatives)

            # Compute precision
            false_positives = np.sum((ground_truth_gray == 0) & (segmentation_mask > 0))
            precision = true_positives / (true_positives + false_positives)

            # Compute accuracy
            true_negatives = np.sum((ground_truth_gray == 0) & (segmentation_mask == 0))
            accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

        dice_scores.append(dice_score)
        jaccard_scores.append(jaccard_score)
        sensitivity_scores.append(sensitivity)
        precision_scores.append(precision)
        accuracy_scores.append(accuracy)

        # Save some images for visualization
        if idx < num_visualizations:
            visualized_images.append((image, ground_truth_gray, segmentation_mask))

    else:
        # Process background-only images
        with torch.no_grad():
            image_embedding = medsam_model.image_encoder(img_1024_tensor) 

        # Full image segmentation for background-only images
        medsam_seg = medsam_inference(medsam_model, image_embedding, None, H, W) 

        # Post-process to remove small spurious detections
        _, medsam_seg_thresholded = cv2.threshold(medsam_seg, 1, 255, cv2.THRESH_BINARY)
        medsam_seg_thresholded = medsam_seg_thresholded.astype(np.uint8)
        num_labels, labels_im = cv2.connectedComponents(medsam_seg_thresholded)

        # Only keep components larger than 50 pixels, meaning that it does not select background label pixels only, but the rest of masks are taken into account
        for label in range(1, num_labels):
            if np.sum(labels_im == label) < 50:
                medsam_seg[labels_im == label] = 0

        # Validate the output
        assert np.sum(medsam_seg) == 0, "The segmentation output for background-only images should be all zeros after post-processing."

        # Case where both ground truth and segmentation are completely background
        dice_scores.append(1.0)
        jaccard_scores.append(1.0)
        sensitivity_scores.append(1.0)
        precision_scores.append(1.0)
        accuracy_scores.append(1.0)

        # Save some images for visualization
        if idx < num_visualizations:
            visualized_images.append((image, ground_truth_gray, medsam_seg))

# Print average inference time
average_inference_time = np.mean(inference_times)
print(f"Average Inference Time: {average_inference_time:.4f} seconds")

# Calculate throughput (images per second)
total_images_processed = len(images)
throughput = total_images_processed / np.sum(inference_times)
print(f"Throughput: {throughput:.2f} images per second")

# Calculate average scores
average_dice_score = np.mean(dice_scores)
average_jaccard_score = np.mean(jaccard_scores)
average_sensitivity = np.mean(sensitivity_scores)
average_precision = np.mean(precision_scores)
average_accuracy = np.mean(accuracy_scores)

print("Average Dice Score:", average_dice_score)
print("Average Jaccard Index (IoU):", average_jaccard_score)
print("Average Sensitivity (Recall):", average_sensitivity)
print("Average Precision:", average_precision)
print("Average Accuracy:", average_accuracy)

# Display some images with predictions for visual verification
for i, (img, gt, pred) in enumerate(visualized_images):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f'Image {i+1}')
    
    plt.subplot(1, 3, 2)
    plt.imshow(gt, cmap='gray')
    plt.title(f'Ground Truth {i+1}')
    
    plt.subplot(1, 3, 3)
    plt.imshow(pred, cmap='gray')
    plt.title(f'Prediction {i+1}')
    
    plt.show()

# Function to get model size
def get_model_size(model):
    # Save the state dictionary of the model
    torch.save(model.state_dict(), "temp.pth")
    
    # Get the size of the saved state dictionary file
    model_size_MB = os.path.getsize("temp.pth") / (1024 * 1024)  
    
    os.remove("temp.pth")
    
    return model_size_MB

model_size = get_model_size(medsam_model)
print(f"Model size: {model_size:.2f} MB")
